In [26]:
import time
import json
import pandas as pd
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver


def MyProtein_store():
    result = []
    MyProtein_URL = "https://www.myprotein.co.kr/c/nutrition/protein/"
    wd = webdriver.Chrome()
    wd.get(MyProtein_URL)
    time.sleep(1)
    html = wd.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # div -> product-list 상품 리스트 목록들
    # product-card-wrapper -> 개별 상품 카드
    # 상품 명 product-card-wrapper -> div(class=flex flex-col) -> a.text
    # product_list = soup.select('#product_list #product-card-wrapper')
    # 가격 span(class = price font-semibold my-auto text-xl) 안의 text
    product_all = soup.find("product-list", {"id": "product-list-wrapper"})
    product_list = product_all('product-card-wrapper')
    print(f"총 {len(product_list)}개의 아이템을 찾았습니다.")
    count: int = 1
    for item in product_list:
        product_name = ""
        product_price = ""
        product = item.find_all('div', {'class': 'flex flex-col'})
        product_name = product[0].find('a').text
        if product_name == "":
            product_name = "상품명 없음"
        product_price = product[1].find('span', {'class': 'price font-semibold my-auto text-xl'}).text
        if product_price == "":
            product_price = "가격 없음"
        print(f"상품 명 : {product_name} 상품 가격 : {product_price}")
        result.append({"index": count, "ProductName": product_name, "Price": product_price})
        count += 1


    wd.quit()
    return result


In [27]:
def save_to_json(data):
    json_file = "MyProtein.json"
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"JSON 파일 저장 완료 : {json_file}")

def save_to_mariadb(data):
    db = pymysql.connect(
        host="localhost",
        user="webuser",  # 변경 필요
        password="webuser",  # 변경 필요
        database="webdb"
    )
    cursor = db.cursor()

    create_table_query = """
    CREATE TABLE IF NOT EXISTS myprotein_shake (
        id INT AUTO_INCREMENT PRIMARY KEY,
        ProductName VARCHAR(255) NOT NULL,
        Price VARCHAR(255) NOT NULL
    )
    """
    cursor.execute(create_table_query)

    insert_query = """
    INSERT INTO myprotein_shake (ProductName, Price)
    VALUES (%s, %s)
                   """

    try:
        cursor.executemany(insert_query, [(d["ProductName"], d["Price"]) for d in data])
        db.commit()
        print(f"{len(data)}개의 프로틴 쉐이커가 DataBase에 저장되었습니다.")
    except Exception as e:
        db.rollback()
        print(f"데이터 저장 중 오류 발생 : {e}")

    cursor.close()
    db.close()

In [31]:
def main():
    print("🚀 MyProtein 매장 정보 크롤링 시작...")
    store_data = MyProtein_store()
    print(f"store_data 확인 : {store_data} ")
    # 파일 기반 저장,
    print("\n💾 JSON 파일 저장 중...")
    save_to_json(store_data)
    # 데이터베이스 저장.
    print("\n📂 MariaDB 저장 중...")
    save_to_mariadb(store_data)

    print("\n🎉 모든 작업 완료!")

if __name__ == '__main__':
    main()

🚀 MyProtein 매장 정보 크롤링 시작...
총 26개의 아이템을 찾았습니다.
상품 명 :  임팩트 웨이 프로틴  상품 가격 :   discounted price   ₩24,900‎  
상품 명 :  임팩트 웨이 아이솔레이트  상품 가격 :   discounted price   ₩53,900‎  
상품 명 :  클리어 웨이 프로틴  상품 가격 :   discounted price   ₩66,900‎  
상품 명 :  임팩트 웨이 게이너  상품 가격 :   discounted price   ₩52,900‎  
상품 명 :  콜라겐 파우더  상품 가격 :   discounted price   ₩49,400‎  
상품 명 :  콜라겐 프로틴 (구: 가수분해콜라겐펩티드)  상품 가격 :   discounted price   ₩65,900‎  
상품 명 :  임팩트 소이 프로틴  상품 가격 :   discounted price   ₩29,900‎  
상품 명 :  오리진 프로틴  상품 가격 :   discounted price   ₩94,900‎  
상품 명 :  임팩트 다이어트 웨이  상품 가격 :   discounted price   ₩52,900‎  
상품 명 :  에센셜 웨이 프로틴  상품 가격 :   discounted price   ₩53,900‎  
상품 명 :  아침 식사 스무디  상품 가격 :   discounted price   ₩81,900‎  
상품 명 :  임팩트 비건 프로틴  상품 가격 :   discounted price   ₩18,900‎  
상품 명 :  느린 흡수 카제인 (구: 미셀라 카세인 (Micellar Casein))  상품 가격 :   discounted price   ₩59,900‎  
상품 명 :  클리어 웨이 프로틴(샘플)  상품 가격 :   discounted price   ₩4,900‎  
상품 명 :  임팩트 완두콩 프로틴  상품 가격 :   discounted price   ₩35,900‎  
상품 명 :  클